# Gaussian mixture models and Tagalog child-directed nasals

This notebook explores using Gaussian mixture models for modeling formant measurements from nasal consonants produced by Tagalog-speaking mothers speaking to their infants collected by Chandan Narayan's lab. The data was collected longitudinally, though modeling longitudinal factors is out of scope for this notebook.

Like many Austronesian languages, Tagalog has three nasal phonemes: /m, n, ŋ/. It is believed that the contrast between the coronal and dorsal nasal is acoustically marginal (i.e. hard to hear/discern), as well as lexically marginal (i.e., it has low _functional load_). So it is of some interest to quantify the degree to which this can be discriminated automatically on the basis of acoustic measures alone. Future work ought to incorporate biases based on lexical identity as well, as in, e.g., Naomi Feldman's work, but that's out of the scope for this notebook.

For the moment, we use scikit-learn's implementation of Gaussian mixture models (`sklearn.mixtures`). Important hyperparameters include:

* how we define the predictor matrix `X`
* the number of mixtures `n_components`
* the complexity of the covariance matrix `covariance_type`
* whether weights are initialized randomly or via an initial k-means step `init_params`
* parameters controlling convergence: tolerance `tol`, regularization coefficient for the covariance matrix `reg_covar`, and the maximum number of iterations `max_iter`

Ideally we'd optimize hyperparameters using [BIC and a held-out development set](http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html#sphx-glr-auto-examples-mixture-plot-gmm-selection-py) or use [variational inference](http://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html) but that's also out of scope for this notebook.

In [11]:
# I'll use logging for verbose output.

import logging
logging.basicConfig(level=logging.INFO)

In [12]:
# Data frame hacking.

from numpy import stack
from pandas import read_csv
from pandas import DataFrame

In [13]:
# Model fitting.

from sklearn.mixture import GaussianMixture

In [14]:
# Scoring.

from bcubed import ClusterScore
from sklearn.metrics.cluster import normalized_mutual_info_score

In [3]:
# We'll use my zip code as a random seed so that eveything's perfectly repeatable.
SEED = 11215

In [15]:
## Loads the data.

# Pseudorandomly chooses a parent/child infant dyad to model.
FILENAME = "nasals_s03f.csv"

# Reads the CSV into memory using Pandas.
d = read_csv(FILENAME)

# Subsets it: we only model cases where the following vowel is "A".
d = d[d["Following Vowel"] == "A"]

In [16]:
## Constructs the predictor matrix.

# Identifies the columns we'll use.
X_COL_NAMES = ("Juncture F1_Bark", "Juncture F2_Bark", "Juncture F3_Bark")

# Filters out rows with NAs in those columns.
d = d.dropna(axis=0, how='any', subset=X_COL_NAMES)

X = stack((d[col_name].values for col_name in X_COL_NAMES), axis=1)
logging.info("Size of dataset: %d x %d", *X.shape)

INFO:root:Size of dataset: 8059 x 3


In [17]:
## Constructs the prediction array and does some exploratory analysis.

Y_pandas = d["Nasal Segment"]
Y_pandas.value_counts()

N     6209
M     1717
NG     133
Name: Nasal Segment, dtype: int64

In [31]:
## Constructs empty model.

# Positive non-zero integer; smaller values give less complex models.
N_COMPONENTS = 5
# One of: "full", "tied", diag", "spherical"; later values give less complex models.
COVARIANCE_TYPE = "full"

# TODO(kbg): Add the convergence-related hyperparameters here.

model_frequentist = GaussianMixture(n_components=N_COMPONENTS,
                                    covariance_type=COVARIANCE_TYPE,
                                    random_state=SEED)

In [32]:
## Fits the model. Might take a moment.

model_frequentist.fit(X)

# Logs resubstitution statistics. I am getting log-likelihoods in the range [-20, -21]
# when I use a reasonable number of components.
logging.info("Resubstitution log-likelihood: %.3f", model_frequentist.score(X))

INFO:root:Resubstitution log-likelihood: -4.623


In [39]:
## Defines a method for scoring.

def score_clustering(gld, hyp, logger):
  """Logs scoring metrics to the provided logger.

  Args:
    gld: An iterable of cluster labels for the gold clusters.
    hyp: An iterable of cluster labels for the hypothesized clusters, of equal
        length to, and corresponding in order to, the first argument.
  """
  # B-Cubed element-averaged F1.
  logger("B-Cubed F1:\t\t%.3f", ClusterScore(gld, hyp).f1())
  # Normalized mutual information.
  logger("Normalized MI:\t%.3f", normalized_mutual_info_score(gld, hyp))

In [ ]:
## Predicts the components and does some exploratory analysis.

# Gets component IDs for each sample and builds a data frame with the nasal phonemes.
fit_frequentist = DataFrame({"nasal": Y_pandas, "component": model_frequentist.predict(X)})

# Calls the scoring function. This can be a bit slow because my implementation of B-Cubed
# isn't so hot.
score_clustering(fit_frequentist["nasal"], fit_frequentist["component"], logging.info)